In [75]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, ReLU, AveragePooling2D, MaxPooling2D, Flatten, Dense, Input
from tensorflow.keras.datasets import cifar10
from sklearn.metrics import f1_score

In [62]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [4]:
x_train, y_train

(array([[[[ 59,  62,  63],
          [ 43,  46,  45],
          [ 50,  48,  43],
          ...,
          [158, 132, 108],
          [152, 125, 102],
          [148, 124, 103]],
 
         [[ 16,  20,  20],
          [  0,   0,   0],
          [ 18,   8,   0],
          ...,
          [123,  88,  55],
          [119,  83,  50],
          [122,  87,  57]],
 
         [[ 25,  24,  21],
          [ 16,   7,   0],
          [ 49,  27,   8],
          ...,
          [118,  84,  50],
          [120,  84,  50],
          [109,  73,  42]],
 
         ...,
 
         [[208, 170,  96],
          [201, 153,  34],
          [198, 161,  26],
          ...,
          [160, 133,  70],
          [ 56,  31,   7],
          [ 53,  34,  20]],
 
         [[180, 139,  96],
          [173, 123,  42],
          [186, 144,  30],
          ...,
          [184, 148,  94],
          [ 97,  62,  34],
          [ 83,  53,  34]],
 
         [[177, 144, 116],
          [168, 129,  94],
          [179, 142,  87],
   

In [5]:
x_train.shape, x_test.shape

((50000, 32, 32, 3), (10000, 32, 32, 3))

In [63]:
x_train = x_train.astype(np.float32) / 255.0
x_test = x_test.astype(np.float32) / 255.0

In [60]:
import numpy as np

class MySequential:
    def __init__(self):
        self.layers = []

    def add(self, layer):
        self.layers.append(layer)

    def forward(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x

class Conv2DScratch:
    def __init__(self, weight, bias, stride=1, padding=0):
        self.weight = weight
        self.bias = bias
        self.stride = stride
        self.padding = padding

    def pad_input(self, x):
        if self.padding == 0:
            return x
        return np.pad(x, ((0, 0), (self.padding, self.padding), (self.padding, self.padding)), mode='constant')

    def forward(self, x):
        C_out, C_in, kH, kW = self.weight.shape
        x_padded = self.pad_input(x)
        _, H_in, W_in = x.shape
        H_out = (H_in + 2*self.padding - kH) // self.stride + 1
        W_out = (W_in + 2*self.padding - kW) // self.stride + 1

        out = np.zeros((C_out, H_out, W_out))

        for oc in range(C_out):
            for i in range(H_out):
                for j in range(W_out):
                    for ic in range(C_in):
                        h_start = i * self.stride
                        w_start = j * self.stride
                        patch = x_padded[ic, h_start:h_start+kH, w_start:w_start+kW]
                        out[oc, i, j] += np.sum(patch * self.weight[oc, ic])
                    out[oc, i, j] += self.bias[oc]
        return out

class ReLUScratch:
    def forward(self, x):
        return np.maximum(0, x)

class MaxPool2DScratch:
    def forward(self, x):
        # x: (C, H, W)
        C, H, W = x.shape
        out = np.zeros((C, H // 2, W // 2))
        for c in range(C):
            for i in range(0, H, 2):
                for j in range(0, W, 2):
                    out[c, i//2, j//2] = np.max(x[c, i:i+2, j:j+2])
        return out

class AveragePool2DScratch:
    def forward(self, x):
        # x: (C, H, W)
        C, H, W = x.shape
        out = np.zeros((C, H // 2, W // 2))
        for c in range(C):
            for i in range(0, H, 2):
                for j in range(0, W, 2):
                    out[c, i//2, j//2] = np.mean(x[c, i:i+2, j:j+2])
        return out

class FlattenScratch:
    def forward(self, x):
        return x.flatten()

class DenseScratch:
    def __init__(self, weight, bias):
        self.weight = weight
        self.bias = bias

    def forward(self, x):
        return np.dot(self.weight, x) + self.bias

## Filters = 3, padding = 1, kernel-size = 3x3, n_conv_layer = 2, MaxPooling

### Library Inititiation

In [174]:
model = Sequential([
    Input(shape=(32, 32, 3)),
    Conv2D(3, (3, 3), padding='same', name='conv1'),
    ReLU(),
    Conv2D(3, (3, 3), padding='same', name='conv2'),
    ReLU(),
    MaxPooling2D(pool_size=2, strides=2, name='pool'),
    Flatten(),
    Dense(10, activation='softmax', name='fc')
])

model.summary()

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1 (Conv2D)                  │ (None, 32, 32, 3)      │            84 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_25 (ReLU)                 │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2 (Conv2D)                  │ (None, 32, 32, 3)      │            84 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_26 (ReLU)                 │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool (MaxPooling2D)             │ (None, 16, 16, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_10 (Flatten)            │ (None, 768)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc (Dense)                      │ (None, 10)             │         7,690 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,858 (30.70 KB)

 Trainable params: 7,858 (30.70 KB)

 Non-trainable params: 0 (0.00 B)

In [175]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [176]:
model.fit(x_train, y_train, epochs=5, batch_size=64, validation_split=0.1)

Epoch 1/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 38s 53ms/step - accuracy: 0.2901 - loss: 1.9501 - val_accuracy: 0.4430 - val_loss: 1.5682
Epoch 2/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 41s 52ms/step - accuracy: 0.4427 - loss: 1.5648 - val_accuracy: 0.4576 - val_loss: 1.5306
Epoch 3/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 42s 53ms/step - accuracy: 0.4561 - loss: 1.5126 - val_accuracy: 0.4594 - val_loss: 1.5079
Epoch 4/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 41s 53ms/step - accuracy: 0.4741 - loss: 1.4702 - val_accuracy: 0.4808 - val_loss: 1.4635
Epoch 5/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 36s 52ms/step - accuracy: 0.4877 - loss: 1.4361 - val_accuracy: 0.4872 - val_loss: 1.4488


### Library Predictions

In [177]:
predictions = model.predict(x_test)
o_y_pred = np.argmax(predictions, axis=1)

313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step


In [178]:
o_macro_f1 = f1_score(y_test.flatten(), o_y_pred, average='macro')
print("Macro F1-score:", o_macro_f1)

Macro F1-score: 0.46777719439512644


### Scratch Predictions

In [11]:
conv1_w, conv1_b = model.get_layer('conv1').get_weights()
conv2_w, conv2_b = model.get_layer('conv2').get_weights()
fc_w, fc_b = model.get_layer('fc').get_weights()

In [12]:
conv1_w, conv1_w.shape

(array([[[[ 0.20181686,  0.19474499,  0.00804396],
          [-0.11266521, -0.25518844, -0.31679237],
          [ 0.40867725, -0.47322714,  0.36957493]],
 
         [[-0.0821192 , -0.2499137 , -0.3775914 ],
          [ 0.16276164, -0.33007005, -0.15938143],
          [ 0.2908762 , -0.2647806 , -0.01479527]],
 
         [[-0.31781787, -0.00726388, -0.13554287],
          [-0.00971645,  0.3641677 , -0.32769293],
          [ 0.29409412,  0.3871966 ,  0.05439963]]],
 
 
        [[[-0.17280091,  0.12756315,  0.16013134],
          [ 0.15562387,  0.14784586,  0.17921151],
          [ 0.24181925,  0.07690808,  0.16184941]],
 
         [[ 0.23799606, -0.21667224, -0.19326784],
          [ 0.3407795 ,  0.06195481, -0.21412583],
          [-0.16232531, -0.3183983 ,  0.00501129]],
 
         [[ 0.04662353,  0.02961974, -0.06483708],
          [ 0.30988178,  0.2624365 , -0.3721563 ],
          [ 0.23959991,  0.14202787,  0.07830749]]],
 
 
        [[[ 0.06943567,  0.1843286 ,  0.33345637],
       

In [13]:
conv2_w, conv2_w.shape

(array([[[[ 0.03726431, -0.04329776, -0.32589334],
          [-0.29146555, -0.2240454 ,  0.24700452],
          [-0.14667055, -0.23373052,  0.2509729 ]],
 
         [[ 0.2554796 , -0.22173457, -0.11864993],
          [ 0.44950828, -0.26475924,  0.26496837],
          [ 0.4340542 , -0.3946044 ,  0.07627098]],
 
         [[-0.13139462,  0.15514335,  0.18540359],
          [ 0.13218279, -0.02844604,  0.06677617],
          [-0.21880315, -0.26002833,  0.32291237]]],
 
 
        [[[ 0.10768411,  0.0871352 , -0.33612633],
          [-0.17196949, -0.11817883, -0.16816796],
          [-0.2532712 ,  0.17764385, -0.097929  ]],
 
         [[-0.14606182,  0.09871736, -0.08329248],
          [-0.17038411, -0.02135123,  0.21269305],
          [ 0.1871723 ,  0.04938828, -0.0348822 ]],
 
         [[ 0.11526012,  0.15266487, -0.19320324],
          [ 0.70019907,  0.00457225, -0.0241727 ],
          [ 0.29055884,  0.21923181,  0.2424934 ]]],
 
 
        [[[-0.13880196, -0.11337296,  0.26523793],
       

In [14]:
conv1_b, conv1_b.shape

(array([-0.11831132,  0.02476181, -0.00595454], dtype=float32), (3,))

In [15]:
conv2_b, conv2_b.shape

(array([ 0.12033126, -0.1203998 , -0.00382589], dtype=float32), (3,))

In [16]:
fc_w, fc_w.shape

(array([[-0.08995786,  0.04566785,  0.19041502, ..., -0.00058044,
          0.1197348 , -0.07899575],
        [-0.0345566 , -0.03962869, -0.0402116 , ...,  0.08866517,
          0.1311681 ,  0.04759823],
        [ 0.05337271, -0.02347573,  0.06782243, ...,  0.00174745,
          0.05324924,  0.04649727],
        ...,
        [ 0.02796304, -0.04837867,  0.14498526, ...,  0.0217226 ,
         -0.10165522, -0.02030138],
        [-0.05486171,  0.08126017, -0.09379694, ...,  0.11447231,
         -0.08688059,  0.07627547],
        [ 0.31614193, -0.08955158, -0.06384657, ..., -0.16196084,
          0.20120344, -0.11255185]], dtype=float32),
 (768, 10))

In [17]:
fc_b, fc_b.shape

(array([-0.07741845, -0.33600414,  0.34297237,  0.2539658 ,  0.2990938 ,
         0.13127244, -0.12809493, -0.22993518,  0.01215785, -0.33301428],
       dtype=float32),
 (10,))

In [18]:
# Convert conv weights to NCHW
conv1_w_nchw = conv1_w.transpose(3, 2, 0, 1)
conv2_w_nchw = conv2_w.transpose(3, 2, 0, 1)

In [19]:
np.savez("trained_cnn_data.npz",
         conv1_w=conv1_w_nchw, conv1_b=conv1_b,
         conv2_w=conv2_w_nchw, conv2_b=conv2_b,
         fc_w=fc_w.T, fc_b=fc_b)  # fc_w: (10, flattened_size)

In [36]:
data = np.load("trained_cnn_data.npz")

In [45]:
myModel = MySequential()

myModel.add(Conv2DScratch(data["conv1_w"], data["conv1_b"], padding=1))
myModel.add(ReLUScratch())
myModel.add(Conv2DScratch(data["conv2_w"], data["conv2_b"], padding=1))
myModel.add(ReLUScratch())
myModel.add(MaxPool2DScratch())
myModel.add(FlattenScratch())
myModel.add(DenseScratch(data["fc_w"], data["fc_b"]))

In [50]:
y_pred = []

for i in range(len(x_test)):
    img = x_test[i].transpose(2, 0, 1)
    out = myModel.forward(img)
    pred = np.argmax(out)
    y_pred.append(pred)

y_pred = np.array(y_pred)

In [54]:
label_counts = np.bincount(y_pred, minlength=10)

for label, count in enumerate(label_counts):
    print(f"Label {label}: {count} samples")

Label 0: 242 samples
Label 1: 6903 samples
Label 2: 46 samples
Label 3: 139 samples
Label 4: 141 samples
Label 5: 101 samples
Label 6: 16 samples
Label 7: 842 samples
Label 8: 121 samples
Label 9: 1449 samples


In [57]:
macro_f1 = f1_score(y_test, y_pred, average='macro')
print("Macro F1-score:", macro_f1)

Macro F1-score: 0.05977427224581109


## Filters = 3, padding = 1, kernel-size = 3x3, n_conv_layer = 2, AveragePooling

### Library Inititalization

In [179]:
model = Sequential([
    Input(shape=(32, 32, 3)),
    Conv2D(3, (3, 3), padding='same', name='conv1'),
    ReLU(),
    Conv2D(3, (3, 3), padding='same', name='conv2'),
    ReLU(),
    AveragePooling2D(pool_size=2, strides=2, name='pool'),
    Flatten(),
    Dense(10, activation='softmax', name='fc')
])

model.summary()

Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1 (Conv2D)                  │ (None, 32, 32, 3)      │            84 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_27 (ReLU)                 │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2 (Conv2D)                  │ (None, 32, 32, 3)      │            84 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_28 (ReLU)                 │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool (AveragePooling2D)         │ (None, 16, 16, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_11 (Flatten)            │ (None, 768)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc (Dense)                      │ (None, 10)             │         7,690 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,858 (30.70 KB)

 Trainable params: 7,858 (30.70 KB)

 Non-trainable params: 0 (0.00 B)

In [180]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [181]:
model.fit(x_train, y_train, epochs=5, batch_size=64, validation_split=0.1)

Epoch 1/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 39s 54ms/step - accuracy: 0.2718 - loss: 1.9981 - val_accuracy: 0.4220 - val_loss: 1.6409
Epoch 2/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 36s 51ms/step - accuracy: 0.4325 - loss: 1.6072 - val_accuracy: 0.4540 - val_loss: 1.5280
Epoch 3/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 40s 51ms/step - accuracy: 0.4659 - loss: 1.5026 - val_accuracy: 0.4600 - val_loss: 1.4915
Epoch 4/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 42s 53ms/step - accuracy: 0.4864 - loss: 1.4517 - val_accuracy: 0.4824 - val_loss: 1.4496
Epoch 5/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 39s 50ms/step - accuracy: 0.4843 - loss: 1.4403 - val_accuracy: 0.4840 - val_loss: 1.4352


### Library Prediction

In [182]:
predictions = model.predict(x_test)
o_y_pred = np.argmax(predictions, axis=1)

313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step


In [183]:
o_macro_f1 = f1_score(y_test.flatten(), o_y_pred, average='macro')
print("Macro F1-score:", o_macro_f1)

Macro F1-score: 0.47572486486052645


Scratch Prediction

In [78]:
conv1_w, conv1_b = model.get_layer('conv1').get_weights()
conv2_w, conv2_b = model.get_layer('conv2').get_weights()
fc_w, fc_b = model.get_layer('fc').get_weights()

In [79]:
# Convert conv weights to NCHW
conv1_w_nchw = conv1_w.transpose(3, 2, 0, 1)
conv2_w_nchw = conv2_w.transpose(3, 2, 0, 1)

In [80]:
np.savez("trained_cnn_data.npz",
         conv1_w=conv1_w_nchw, conv1_b=conv1_b,
         conv2_w=conv2_w_nchw, conv2_b=conv2_b,
         fc_w=fc_w.T, fc_b=fc_b)  # fc_w: (10, flattened_size)

In [81]:
data = np.load("trained_cnn_data.npz")

In [82]:
myModel = MySequential()

myModel.add(Conv2DScratch(data["conv1_w"], data["conv1_b"], padding=1))
myModel.add(ReLUScratch())
myModel.add(Conv2DScratch(data["conv2_w"], data["conv2_b"], padding=1))
myModel.add(ReLUScratch())
myModel.add(AveragePool2DScratch())
myModel.add(FlattenScratch())
myModel.add(DenseScratch(data["fc_w"], data["fc_b"]))

In [83]:
y_pred = []

for i in range(len(x_test)):
    img = x_test[i].transpose(2, 0, 1)
    out = myModel.forward(img)
    pred = np.argmax(out)
    y_pred.append(pred)

y_pred = np.array(y_pred)

In [84]:
label_counts = np.bincount(y_pred, minlength=10)

for label, count in enumerate(label_counts):
    print(f"Label {label}: {count} samples")

Label 0: 1276 samples
Label 1: 2038 samples
Label 2: 9 samples
Label 3: 15 samples
Label 4: 0 samples
Label 5: 1 samples
Label 6: 0 samples
Label 7: 0 samples
Label 8: 5016 samples
Label 9: 1645 samples


In [85]:
macro_f1 = f1_score(y_test, y_pred, average='macro')
print("Macro F1-score:", macro_f1)

Macro F1-score: 0.06641523821151936


## Filters = 3, padding = 1, kernel-size = 3x3, n_conv_layer = 3, MaxPooling

### Library Initiation

In [184]:
model = Sequential([
    Input(shape=(32, 32, 3)),
    Conv2D(3, (3, 3), padding='same', name='conv1'),
    ReLU(),
    Conv2D(3, (3, 3), padding='same', name='conv2'),
    ReLU(),
    Conv2D(3, (3, 3), padding='same', name='conv3'),
    ReLU(),
    MaxPooling2D(pool_size=2, strides=2, name='pool'),
    Flatten(),
    Dense(10, activation='softmax', name='fc')
])

model.summary()

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1 (Conv2D)                  │ (None, 32, 32, 3)      │            84 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_29 (ReLU)                 │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2 (Conv2D)                  │ (None, 32, 32, 3)      │            84 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_30 (ReLU)                 │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3 (Conv2D)                  │ (None, 32, 32, 3)      │            84 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_31 (ReLU)                 │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool (MaxPooling2D)             │ (None, 16, 16, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_12 (Flatten)            │ (None, 768)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc (Dense)                      │ (None, 10)             │         7,690 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,942 (31.02 KB)

 Trainable params: 7,942 (31.02 KB)

 Non-trainable params: 0 (0.00 B)

In [185]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=5, batch_size=64, validation_split=0.1)

Epoch 1/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 57s 78ms/step - accuracy: 0.2577 - loss: 2.0239 - val_accuracy: 0.4198 - val_loss: 1.6427
Epoch 2/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 57s 81ms/step - accuracy: 0.4255 - loss: 1.6366 - val_accuracy: 0.4634 - val_loss: 1.5328
Epoch 3/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 81s 80ms/step - accuracy: 0.4669 - loss: 1.5302 - val_accuracy: 0.4848 - val_loss: 1.4725
Epoch 4/5
475/704 ━━━━━━━━━━━━━━━━━━━━ 17s 75ms/step - accuracy: 0.4932 - loss: 1.4560

### Library Predictions

In [ ]:
predictions = model.predict(x_test)
o_y_pred = np.argmax(predictions, axis=1)

In [ ]:
o_macro_f1 = f1_score(y_test.flatten(), o_y_pred, average='macro')
print("Macro F1-score:", o_macro_f1)

### Scratch Predictions

In [95]:
conv1_w, conv1_b = model.get_layer('conv1').get_weights()
conv2_w, conv2_b = model.get_layer('conv2').get_weights()
conv3_w, conv3_b = model.get_layer('conv3').get_weights()
fc_w, fc_b = model.get_layer('fc').get_weights()

In [96]:
# Convert conv weights to NCHW
conv1_w_nchw = conv1_w.transpose(3, 2, 0, 1)
conv2_w_nchw = conv2_w.transpose(3, 2, 0, 1)
conv3_w_nchw = conv3_w.transpose(3, 2, 0, 1)

In [97]:
np.savez("trained_cnn_data.npz",
         conv1_w=conv1_w_nchw, conv1_b=conv1_b,
         conv2_w=conv2_w_nchw, conv2_b=conv2_b,
         conv3_w=conv3_w_nchw, conv3_b=conv3_b,
         fc_w=fc_w.T, fc_b=fc_b)  # fc_w: (10, flattened_size)

In [98]:
data = np.load("trained_cnn_data.npz")

In [101]:
myModel = MySequential()

myModel.add(Conv2DScratch(data["conv1_w"], data["conv1_b"], padding=1))
myModel.add(ReLUScratch())
myModel.add(Conv2DScratch(data["conv2_w"], data["conv2_b"], padding=1))
myModel.add(ReLUScratch())
myModel.add(Conv2DScratch(data["conv3_w"], data["conv3_b"], padding=1))
myModel.add(ReLUScratch())
myModel.add(MaxPool2DScratch())
myModel.add(FlattenScratch())
myModel.add(DenseScratch(data["fc_w"], data["fc_b"]))

In [102]:
y_pred = []

for i in range(len(x_test)):
    img = x_test[i].transpose(2, 0, 1)
    out = myModel.forward(img)
    pred = np.argmax(out)
    y_pred.append(pred)

y_pred = np.array(y_pred)

In [103]:
label_counts = np.bincount(y_pred, minlength=10)

for label, count in enumerate(label_counts):
    print(f"Label {label}: {count} samples")

Label 0: 34 samples
Label 1: 3368 samples
Label 2: 496 samples
Label 3: 264 samples
Label 4: 1074 samples
Label 5: 3878 samples
Label 6: 452 samples
Label 7: 395 samples
Label 8: 12 samples
Label 9: 27 samples


In [104]:
macro_f1 = f1_score(y_test, y_pred, average='macro')
print("Macro F1-score:", macro_f1)

Macro F1-score: 0.09394425108571784


## Filters = 3, padding = 1, kernel-size = 3x3, n_conv_layer = 4, MaxPooling

### Library Initiation

In [ ]:
model = Sequential([
    Input(shape=(32, 32, 3)),
    Conv2D(3, (3, 3), padding='same', name='conv1'),
    ReLU(),
    Conv2D(3, (3, 3), padding='same', name='conv2'),
    ReLU(),
    Conv2D(3, (3, 3), padding='same', name='conv3'),
    ReLU(),
    Conv2D(3, (3, 3), padding='same', name='conv4'),
    ReLU(),
    MaxPooling2D(pool_size=2, strides=2, name='pool'),
    Flatten(),
    Dense(10, activation='softmax', name='fc')
])

model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=5, batch_size=64, validation_split=0.1)

### Library Predictions

In [ ]:
predictions = model.predict(x_test)
o_y_pred = np.argmax(predictions, axis=1)

In [ ]:
o_macro_f1 = f1_score(y_test.flatten(), o_y_pred, average='macro')
print("Macro F1-score:", o_macro_f1)

### Scratch Predictions

In [110]:
conv1_w, conv1_b = model.get_layer('conv1').get_weights()
conv2_w, conv2_b = model.get_layer('conv2').get_weights()
conv3_w, conv3_b = model.get_layer('conv3').get_weights()
conv4_w, conv4_b = model.get_layer('conv4').get_weights()
fc_w, fc_b = model.get_layer('fc').get_weights()

In [111]:
# Convert conv weights to NCHW
conv1_w_nchw = conv1_w.transpose(3, 2, 0, 1)
conv2_w_nchw = conv2_w.transpose(3, 2, 0, 1)
conv3_w_nchw = conv3_w.transpose(3, 2, 0, 1)
conv4_w_nchw = conv4_w.transpose(3, 2, 0, 1)

In [112]:
np.savez("trained_cnn_data.npz",
         conv1_w=conv1_w_nchw, conv1_b=conv1_b,
         conv2_w=conv2_w_nchw, conv2_b=conv2_b,
         conv3_w=conv3_w_nchw, conv3_b=conv3_b,
         conv4_w=conv4_w_nchw, conv4_b=conv4_b,
         fc_w=fc_w.T, fc_b=fc_b)  # fc_w: (10, flattened_size)

In [113]:
data = np.load("trained_cnn_data.npz")

In [114]:
myModel = MySequential()

myModel.add(Conv2DScratch(data["conv1_w"], data["conv1_b"], padding=1))
myModel.add(ReLUScratch())
myModel.add(Conv2DScratch(data["conv2_w"], data["conv2_b"], padding=1))
myModel.add(ReLUScratch())
myModel.add(Conv2DScratch(data["conv3_w"], data["conv3_b"], padding=1))
myModel.add(ReLUScratch())
myModel.add(Conv2DScratch(data["conv4_w"], data["conv4_b"], padding=1))
myModel.add(ReLUScratch())
myModel.add(MaxPool2DScratch())
myModel.add(FlattenScratch())
myModel.add(DenseScratch(data["fc_w"], data["fc_b"]))

In [115]:
y_pred = []

for i in range(len(x_test)):
    img = x_test[i].transpose(2, 0, 1)
    out = myModel.forward(img)
    pred = np.argmax(out)
    y_pred.append(pred)

y_pred = np.array(y_pred)

In [116]:
label_counts = np.bincount(y_pred, minlength=10)

for label, count in enumerate(label_counts):
    print(f"Label {label}: {count} samples")

Label 0: 2861 samples
Label 1: 1391 samples
Label 2: 421 samples
Label 3: 24 samples
Label 4: 858 samples
Label 5: 4 samples
Label 6: 1433 samples
Label 7: 1353 samples
Label 8: 10 samples
Label 9: 1645 samples


In [117]:
macro_f1 = f1_score(y_test, y_pred, average='macro')
print("Macro F1-score:", macro_f1)

Macro F1-score: 0.05810960031865836


## Filters = 2, padding = 1, kernel-size = 3x3, n_conv_layer = 2, MaxPooling

### Library Initialization

In [ ]:
model = Sequential([
    Input(shape=(32, 32, 3)),
    Conv2D(2, (3, 3), padding='same', name='conv1'),
    ReLU(),
    Conv2D(2, (3, 3), padding='same', name='conv2'),
    ReLU(),
    MaxPooling2D(pool_size=2, strides=2, name='pool'),
    Flatten(),
    Dense(10, activation='softmax', name='fc')
])

model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=5, batch_size=64, validation_split=0.1)

### Library Predictions

In [ ]:
predictions = model.predict(x_test)
o_y_pred = np.argmax(predictions, axis=1)

In [ ]:
o_macro_f1 = f1_score(y_test.flatten(), o_y_pred, average='macro')
print("Macro F1-score:", o_macro_f1)

### Scratch Predictions

In [123]:
conv1_w, conv1_b = model.get_layer('conv1').get_weights()
conv2_w, conv2_b = model.get_layer('conv2').get_weights()
fc_w, fc_b = model.get_layer('fc').get_weights()

In [124]:
# Convert conv weights to NCHW
conv1_w_nchw = conv1_w.transpose(3, 2, 0, 1)
conv2_w_nchw = conv2_w.transpose(3, 2, 0, 1)

In [125]:
np.savez("trained_cnn_data.npz",
         conv1_w=conv1_w_nchw, conv1_b=conv1_b,
         conv2_w=conv2_w_nchw, conv2_b=conv2_b,
         fc_w=fc_w.T, fc_b=fc_b)  # fc_w: (10, flattened_size)

In [126]:
data = np.load("trained_cnn_data.npz")

In [127]:
myModel = MySequential()

myModel.add(Conv2DScratch(data["conv1_w"], data["conv1_b"], padding=1))
myModel.add(ReLUScratch())
myModel.add(Conv2DScratch(data["conv2_w"], data["conv2_b"], padding=1))
myModel.add(ReLUScratch())
myModel.add(MaxPool2DScratch())
myModel.add(FlattenScratch())
myModel.add(DenseScratch(data["fc_w"], data["fc_b"]))

In [128]:
y_pred = []

for i in range(len(x_test)):
    img = x_test[i].transpose(2, 0, 1)
    out = myModel.forward(img)
    pred = np.argmax(out)
    y_pred.append(pred)

y_pred = np.array(y_pred)

In [129]:
label_counts = np.bincount(y_pred, minlength=10)

for label, count in enumerate(label_counts):
    print(f"Label {label}: {count} samples")

Label 0: 6 samples
Label 1: 262 samples
Label 2: 55 samples
Label 3: 9467 samples
Label 4: 0 samples
Label 5: 14 samples
Label 6: 143 samples
Label 7: 1 samples
Label 8: 17 samples
Label 9: 35 samples


In [130]:
macro_f1 = f1_score(y_test, y_pred, average='macro')
print("Macro F1-score:", macro_f1)

Macro F1-score: 0.03572897797601249


## Filters = 4, padding = 1, kernel-size = 3x3, n_conv_layer = 2, MaxPooling

### Library Initialization

In [ ]:
model = Sequential([
    Input(shape=(32, 32, 3)),
    Conv2D(4, (3, 3), padding='same', name='conv1'),
    ReLU(),
    Conv2D(4, (3, 3), padding='same', name='conv2'),
    ReLU(),
    MaxPooling2D(pool_size=2, strides=2, name='pool'),
    Flatten(),
    Dense(10, activation='softmax', name='fc')
])

model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=5, batch_size=64, validation_split=0.1)

### Library Predictions

In [ ]:
predictions = model.predict(x_test)
o_y_pred = np.argmax(predictions, axis=1)

In [ ]:
o_macro_f1 = f1_score(y_test.flatten(), o_y_pred, average='macro')
print("Macro F1-score:", o_macro_f1)

### Scratch Predictions

In [136]:
conv1_w, conv1_b = model.get_layer('conv1').get_weights()
conv2_w, conv2_b = model.get_layer('conv2').get_weights()
fc_w, fc_b = model.get_layer('fc').get_weights()

In [137]:
# Convert conv weights to NCHW
conv1_w_nchw = conv1_w.transpose(3, 2, 0, 1)
conv2_w_nchw = conv2_w.transpose(3, 2, 0, 1)

In [138]:
np.savez("trained_cnn_data.npz",
         conv1_w=conv1_w_nchw, conv1_b=conv1_b,
         conv2_w=conv2_w_nchw, conv2_b=conv2_b,
         fc_w=fc_w.T, fc_b=fc_b)  # fc_w: (10, flattened_size)

In [139]:
data = np.load("trained_cnn_data.npz")

In [140]:
myModel = MySequential()

myModel.add(Conv2DScratch(data["conv1_w"], data["conv1_b"], padding=1))
myModel.add(ReLUScratch())
myModel.add(Conv2DScratch(data["conv2_w"], data["conv2_b"], padding=1))
myModel.add(ReLUScratch())
myModel.add(MaxPool2DScratch())
myModel.add(FlattenScratch())
myModel.add(DenseScratch(data["fc_w"], data["fc_b"]))

In [141]:
y_pred = []

for i in range(len(x_test)):
    img = x_test[i].transpose(2, 0, 1)
    out = myModel.forward(img)
    pred = np.argmax(out)
    y_pred.append(pred)

y_pred = np.array(y_pred)

In [142]:
label_counts = np.bincount(y_pred, minlength=10)

for label, count in enumerate(label_counts):
    print(f"Label {label}: {count} samples")

Label 0: 3291 samples
Label 1: 4370 samples
Label 2: 1 samples
Label 3: 6 samples
Label 4: 3 samples
Label 5: 0 samples
Label 6: 121 samples
Label 7: 827 samples
Label 8: 1200 samples
Label 9: 181 samples


In [143]:
macro_f1 = f1_score(y_test, y_pred, average='macro')
print("Macro F1-score:", macro_f1)

Macro F1-score: 0.06162947789160027


## Filters = 3, padding = 2, kernel-size = 5x5, n_conv_layer = 2, MaxPooling

### Library Initialization

In [144]:
model = Sequential([
    Input(shape=(32, 32, 3)),
    Conv2D(3, (5, 5), padding='same', name='conv1'),
    ReLU(),
    Conv2D(3, (5, 5), padding='same', name='conv2'),
    ReLU(),
    MaxPooling2D(pool_size=2, strides=2, name='pool'),
    Flatten(),
    Dense(10, activation='softmax', name='fc')
])

model.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1 (Conv2D)                  │ (None, 32, 32, 3)      │           228 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_21 (ReLU)                 │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2 (Conv2D)                  │ (None, 32, 32, 3)      │           228 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_22 (ReLU)                 │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool (MaxPooling2D)             │ (None, 16, 16, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_8 (Flatten)             │ (None, 768)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc (Dense)                      │ (None, 10)             │         7,690 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,146 (31.82 KB)

 Trainable params: 8,146 (31.82 KB)

 Non-trainable params: 0 (0.00 B)

In [145]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [146]:
model.fit(x_train, y_train, epochs=5, batch_size=64, validation_split=0.1)

Epoch 1/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 72s 101ms/step - accuracy: 0.2918 - loss: 1.9466 - val_accuracy: 0.4106 - val_loss: 1.6551
Epoch 2/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 80s 99ms/step - accuracy: 0.4392 - loss: 1.5775 - val_accuracy: 0.4484 - val_loss: 1.5260
Epoch 3/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 83s 100ms/step - accuracy: 0.4656 - loss: 1.5073 - val_accuracy: 0.4798 - val_loss: 1.4638
Epoch 4/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 81s 99ms/step - accuracy: 0.4790 - loss: 1.4649 - val_accuracy: 0.4712 - val_loss: 1.4880
Epoch 5/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 71s 101ms/step - accuracy: 0.4833 - loss: 1.4538 - val_accuracy: 0.4876 - val_loss: 1.4349


### Library Predictions

In [147]:
predictions = model.predict(x_test)
o_y_pred = np.argmax(predictions, axis=1)

313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 20ms/step


In [148]:
o_macro_f1 = f1_score(y_test.flatten(), o_y_pred, average='macro')
print("Macro F1-score:", o_macro_f1)

Macro F1-score: 0.47825036080672445


### Scratch Predictions

In [149]:
conv1_w, conv1_b = model.get_layer('conv1').get_weights()
conv2_w, conv2_b = model.get_layer('conv2').get_weights()
fc_w, fc_b = model.get_layer('fc').get_weights()

In [150]:
# Convert conv weights to NCHW
conv1_w_nchw = conv1_w.transpose(3, 2, 0, 1)
conv2_w_nchw = conv2_w.transpose(3, 2, 0, 1)

In [151]:
np.savez("trained_cnn_data.npz",
         conv1_w=conv1_w_nchw, conv1_b=conv1_b,
         conv2_w=conv2_w_nchw, conv2_b=conv2_b,
         fc_w=fc_w.T, fc_b=fc_b)  # fc_w: (10, flattened_size)

In [152]:
data = np.load("trained_cnn_data.npz")

In [157]:
myModel = MySequential()

myModel.add(Conv2DScratch(data["conv1_w"], data["conv1_b"], padding=2))
myModel.add(ReLUScratch())
myModel.add(Conv2DScratch(data["conv2_w"], data["conv2_b"], padding=2))
myModel.add(ReLUScratch())
myModel.add(MaxPool2DScratch())
myModel.add(FlattenScratch())
myModel.add(DenseScratch(data["fc_w"], data["fc_b"]))

In [158]:
y_pred = []

for i in range(len(x_test)):
    img = x_test[i].transpose(2, 0, 1)
    out = myModel.forward(img)
    pred = np.argmax(out)
    y_pred.append(pred)

y_pred = np.array(y_pred)

In [159]:
label_counts = np.bincount(y_pred, minlength=10)

for label, count in enumerate(label_counts):
    print(f"Label {label}: {count} samples")

Label 0: 82 samples
Label 1: 7479 samples
Label 2: 221 samples
Label 3: 1089 samples
Label 4: 0 samples
Label 5: 151 samples
Label 6: 0 samples
Label 7: 4 samples
Label 8: 3 samples
Label 9: 971 samples


In [160]:
macro_f1 = f1_score(y_test, y_pred, average='macro')
print("Macro F1-score:", macro_f1)

Macro F1-score: 0.05260317303743369


## Filters = 3, padding = 1, kernel-size = 7x7, n_conv_layer = 2, MaxPooling

### Library Initialization

In [161]:
model = Sequential([
    Input(shape=(32, 32, 3)),
    Conv2D(3, (7, 7), padding='same', name='conv1'),
    ReLU(),
    Conv2D(3, (7, 7), padding='same', name='conv2'),
    ReLU(),
    MaxPooling2D(pool_size=2, strides=2, name='pool'),
    Flatten(),
    Dense(10, activation='softmax', name='fc')
])

model.summary()

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1 (Conv2D)                  │ (None, 32, 32, 3)      │           444 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_23 (ReLU)                 │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2 (Conv2D)                  │ (None, 32, 32, 3)      │           444 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_24 (ReLU)                 │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool (MaxPooling2D)             │ (None, 16, 16, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_9 (Flatten)             │ (None, 768)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc (Dense)                      │ (None, 10)             │         7,690 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,578 (33.51 KB)

 Trainable params: 8,578 (33.51 KB)

 Non-trainable params: 0 (0.00 B)

In [162]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [163]:
model.fit(x_train, y_train, epochs=5, batch_size=64, validation_split=0.1)

Epoch 1/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 119s 167ms/step - accuracy: 0.2697 - loss: 2.0070 - val_accuracy: 0.4188 - val_loss: 1.6414
Epoch 2/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 140s 165ms/step - accuracy: 0.4273 - loss: 1.6161 - val_accuracy: 0.4414 - val_loss: 1.5561
Epoch 3/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 117s 167ms/step - accuracy: 0.4537 - loss: 1.5397 - val_accuracy: 0.4470 - val_loss: 1.5204
Epoch 4/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 140s 164ms/step - accuracy: 0.4650 - loss: 1.5057 - val_accuracy: 0.4772 - val_loss: 1.4860
Epoch 5/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 143s 166ms/step - accuracy: 0.4834 - loss: 1.4609 - val_accuracy: 0.4622 - val_loss: 1.5598


### Library Predictions

In [164]:
predictions = model.predict(x_test)
o_y_pred = np.argmax(predictions, axis=1)

313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 30ms/step


In [165]:
o_macro_f1 = f1_score(y_test.flatten(), o_y_pred, average='macro')
print("Macro F1-score:", o_macro_f1)

Macro F1-score: 0.433556296723384


### Scratch Predictions

In [166]:
conv1_w, conv1_b = model.get_layer('conv1').get_weights()
conv2_w, conv2_b = model.get_layer('conv2').get_weights()
fc_w, fc_b = model.get_layer('fc').get_weights()

In [167]:
# Convert conv weights to NCHW
conv1_w_nchw = conv1_w.transpose(3, 2, 0, 1)
conv2_w_nchw = conv2_w.transpose(3, 2, 0, 1)

In [168]:
np.savez("trained_cnn_data.npz",
         conv1_w=conv1_w_nchw, conv1_b=conv1_b,
         conv2_w=conv2_w_nchw, conv2_b=conv2_b,
         fc_w=fc_w.T, fc_b=fc_b)  # fc_w: (10, flattened_size)

In [169]:
data = np.load("trained_cnn_data.npz")

In [170]:
myModel = MySequential()

myModel.add(Conv2DScratch(data["conv1_w"], data["conv1_b"], padding=3))
myModel.add(ReLUScratch())
myModel.add(Conv2DScratch(data["conv2_w"], data["conv2_b"], padding=3))
myModel.add(ReLUScratch())
myModel.add(MaxPool2DScratch())
myModel.add(FlattenScratch())
myModel.add(DenseScratch(data["fc_w"], data["fc_b"]))

In [171]:
y_pred = []

for i in range(len(x_test)):
    img = x_test[i].transpose(2, 0, 1)
    out = myModel.forward(img)
    pred = np.argmax(out)
    y_pred.append(pred)

y_pred = np.array(y_pred)

In [172]:
label_counts = np.bincount(y_pred, minlength=10)

for label, count in enumerate(label_counts):
    print(f"Label {label}: {count} samples")

Label 0: 0 samples
Label 1: 0 samples
Label 2: 91 samples
Label 3: 2336 samples
Label 4: 0 samples
Label 5: 7553 samples
Label 6: 19 samples
Label 7: 1 samples
Label 8: 0 samples
Label 9: 0 samples


In [173]:
macro_f1 = f1_score(y_test, y_pred, average='macro')
print("Macro F1-score:", macro_f1)

Macro F1-score: 0.028861659882896913
